# JOB OFFER'S DATA (part 3: cleaning and transforming)

## 1. SETTING UP

In [1]:
# Load nessesary libraries.
import pandas as pd
import re
from random import randint
from datetime import datetime, timedelta
import os

In [2]:
# Define global variables.
label_value = ['registered_nurse', 'electrician', 'data_analyst', 'registered_nurse', 'electrician', 'data_analyst']
csv_file_name = ['data_jobads_rn.csv', 'data_jobads_e.csv', 'data_jobads_da.csv', 'data_jobads_rn_20jan.csv', 'data_jobads_e_20jan.csv', 'data_jobads_da_20jan.csv']
data_frame = ['df1', 'df2', 'df3', 'df4', 'df5', 'df6']
to_remove = ['salary', 'schedule', 'benefit', 'location', 'job type', 'office', 'tag', 'employment type', 'email', 'ref.no', 
             'contact name', 'job ref', 'offer in return', 'job title', 'received by', 'signature date', '______', 'block capitals']
date_of_download = ['January 10, 2024', 'January 20, 2024']
before_30_days = ['before December 11, 2023', 'before December 21, 2023']
directory = r'C:\Users\temulenbd\OneDrive\Desktop\learn\github_repo\cct\capstone_project'
df = pd.DataFrame()

In [3]:
# Define a custom function to import a CSV file, add a label column, and return the DataFrame.
def import_and_label(csv, value):
    
    # Declare global variables.
    global df
    
    # Read the CSV file into a DataFrame.
    new_df = pd.read_csv(csv, index_col=None)
    
    # Add a new column 'label' with the specified value.
    new_df['label'] = value
    
    # Concatenate the existing DataFrame 'df' with the new DataFrame.
    df = pd.concat([df, new_df], ignore_index=True)
    
    print('The values of the <'+ csv + '> file were successfully added to the <df>.')

In [4]:
# Define a custom function to replace 'Just posted' or 'Today' with 'January 10, 2024'.
def replace_just_posted_10th(date):
    global date_of_download
    date_download = date_of_download[0]
    return date_download if 'Just posted' in str(date) or 'Today' in str(date) else date

# Define a custom function to replace 'Just posted' or 'Today' with 'January 20, 2024'.
def replace_just_posted_20th(date):
    global date_of_download
    date_download = date_of_download[1]
    return date_download if 'Just posted' in str(date) or 'Today' in str(date) else date

In [5]:
# Define a custom function to extract and replace values with relevant information.
def remove_replace_elements_10th(date):
    global date_of_download
    if date and ('day' in str(date) or 'days' in str(date)):
        days_ago = int(re.search(r'(\d+) (?:day|days) ago', str(date)).group(1))
        new_date = datetime.strptime(date_of_download[0], '%B %d, %Y') - timedelta(days=days_ago)
        return new_date.strftime('%B %d, %Y')
    else:
        return date
    
# Define a custom function extract and replace with relevant information.    
def remove_replace_elements_20th(date):
    global date_of_download
    if date and ('day' in str(date) or 'days' in str(date)):
        days_ago = int(re.search(r'(\d+) (?:day|days) ago', str(date)).group(1))
        new_date = datetime.strptime(date_of_download[1], '%B %d, %Y') - timedelta(days=days_ago)
        return new_date.strftime('%B %d, %Y')
    else:
        return date

In [6]:
# Define the costum function to find and remove unnecessary or private information.
def remove_elements_with_colon(column_value, remove_value):
        
    # Convert the value to lowercase.
    column_value = column_value.lower()
    
    # Split each value of the column into a list using '\n' as a separator.
    elements = column_value.split('\n')
    
    # Find the index of the element containing given value.
    index_of_element = next((i for i, elements in enumerate(elements) if remove_value in elements and ':' in elements), None)
    
    # Check if the first conditions are present in the text.
    if index_of_element is not None:
        next_colon_index = next((j for j in range(index_of_element + 1, len(elements)) if ':' in elements[j]), None)
        
        # Add an extra condition to check the second conditions are present in the text.
        if next_colon_index is not None:
            del elements[index_of_element:next_colon_index]
            return '\n'.join(elements)
        else:
            return column_value
        
    else:
        return column_value

In [7]:
# Define the costum function to find and remove unnecessary or private information.
def remove_unnec_lines(column_value, remove_value):
    # Split each value of the column into a list using '\n' as a separator.
    elements = column_value.split('\n')

    # Function to check if a line contains an email address, phone number, or link
    def is_unwanted_line(line):
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        phone_pattern = r'\b\d+[-.\s+]?\d+[-.\s+]?\d+\b'
        link_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

        return re.search(email_pattern, line) or re.search(phone_pattern, line) or re.search(link_pattern, line)

    # Remove lines containing the specified remove_value
    index_of_element = next((i for i, element in enumerate(elements) if remove_value in element), None)
    if index_of_element is not None:
        elements.pop(index_of_element)

    # Remove lines containing emails, phone numbers, and links
    elements = [element for element in elements if not is_unwanted_line(element)]

    return '\n'.join(elements)


## 1. CLEANING AND TRANSFORMATION

**january 10, 2024**

In [8]:
# Merge csv file from 10th of January into one DataFrame.
for x in range(3):
    import_and_label(csv_file_name[x], label_value[x])

print('--------------------------------')
rows = df.shape[0]
columns = df.shape[1]
print(f'The newly created <df> has {rows} rows and {columns} columns.')

The values of the <data_jobads_rn.csv> file were successfully added to the <df>.
The values of the <data_jobads_e.csv> file were successfully added to the <df>.
The values of the <data_jobads_da.csv> file were successfully added to the <df>.
--------------------------------
The newly created <df> has 1034 rows and 6 columns.


In [9]:
# Remove duplicate rows based on the 'id' column and keep the original rows.
df = df.drop_duplicates(subset='id', keep='first')

print(f'There are {rows-df.shape[0]} duplicate rows were removed from the <df>')
print(f'Now, the <df> contains {df.shape[0]} rows and {df.shape[1]} columns.')

There are 114 duplicate rows were removed from the <df>
Now, the <df> contains 920 rows and 6 columns.


In [10]:
# Extract and replace values of the 'date' column.
df['date'] = df['date'].replace(['not available', 'Hiring ongoing', None], 'unknown').replace(['Posted 30+ days ago'], before_30_days[0]).apply(replace_just_posted_10th).apply(remove_replace_elements_10th)

# Save the values and reset the <df>.
df_10th = df.copy()
df = pd.DataFrame()

# Assuming df is your DataFrame and 'date' is the column
date_counts = df_10th['date'].value_counts()

# Print the counts for each unique value in the 'date' column
print(date_counts)

before December 11, 2023    425
unknown                      72
January 08, 2024             71
December 20, 2023            44
January 05, 2024             40
December 22, 2023            37
January 10, 2024             30
January 09, 2024             28
January 03, 2024             28
January 04, 2024             21
January 06, 2024             18
December 13, 2023            14
December 19, 2023            12
December 14, 2023            10
December 21, 2023            10
December 23, 2023             8
December 12, 2023             8
December 16, 2023             7
December 30, 2023             6
January 02, 2024              6
December 15, 2023             5
January 07, 2024              5
December 29, 2023             3
December 18, 2023             2
January 01, 2024              2
December 28, 2023             2
December 26, 2023             2
December 31, 2023             1
December 11, 2023             1
December 24, 2023             1
December 27, 2023             1
Name: da

**january 20, 2024**

In [11]:
# Merge all csv file into one DataFrame
for x in range(3, 6):
    import_and_label(csv_file_name[x], label_value[x])

print('--------------------------------')
rows = df.shape[0]
columns = df.shape[1]
print(f'The newly created <df> has {rows} rows and {columns} columns.')

The values of the <data_jobads_rn_20jan.csv> file were successfully added to the <df>.
The values of the <data_jobads_e_20jan.csv> file were successfully added to the <df>.
The values of the <data_jobads_da_20jan.csv> file were successfully added to the <df>.
--------------------------------
The newly created <df> has 333 rows and 6 columns.


In [12]:
# Remove duplicate rows based on the 'id' column and keep the original rows.
df = df.drop_duplicates(subset='id', keep='first')

print(f'There are {rows-df.shape[0]} duplicate rows were removed from the <df>')
print(f'Now, the <df> contains {df.shape[0]} rows and {df.shape[1]} columns.')

There are 14 duplicate rows were removed from the <df>
Now, the <df> contains 319 rows and 6 columns.


In [13]:
# Extract and replace values of the 'date' column.
df['date'] = df['date'].replace(['not available', 'Hiring ongoing', None], 'unknown').replace(['Posted 30+ days ago'], before_30_days[1]).apply(replace_just_posted_20th).apply(remove_replace_elements_20th)

# Save the values and reset the <df>.
df_20th = df.copy()
df = pd.DataFrame()

# Assuming df is your DataFrame and 'date' is the column
date_counts = df_20th['date'].value_counts()

# Print the counts for each unique value in the 'date' column
print(date_counts)

January 18, 2024            61
unknown                     40
before December 21, 2023    35
January 20, 2024            32
January 17, 2024            30
January 19, 2024            29
January 16, 2024            26
January 12, 2024            19
January 13, 2024            14
January 11, 2024             9
January 10, 2024             8
January 15, 2024             7
January 09, 2024             3
January 14, 2024             1
December 22, 2023            1
January 06, 2024             1
January 02, 2024             1
January 03, 2024             1
December 23, 2023            1
Name: date, dtype: int64


**combining and finalizing the process in merged data frame**

In [14]:
# Merge two dataframes into one.
df = pd.concat([df_10th, df_20th], ignore_index=True)
rows = df.shape[0]
columns = df.shape[1]

print(f'The newly created <df> has {rows} rows and {columns} columns.')

The newly created <df> has 1239 rows and 6 columns.


In [15]:
# Remove duplicate rows based on the 'id' column and keep the original rows.
df = df.drop_duplicates(subset='link', keep='first')

print(f'There are {rows-df.shape[0]} duplicate rows were removed from the <df>')
print(f'Now, the <df> contains {df.shape[0]} rows and {df.shape[1]} columns.')

There are 0 duplicate rows were removed from the <df>
Now, the <df> contains 1239 rows and 6 columns.


In [16]:
# Remove any private and unnecessary information from the job details.
for val_rem in to_remove:
    df['job_description'] = df['job_description'].apply(lambda x: remove_elements_with_colon(x, val_rem)
                                                        ).apply(lambda x:remove_unnec_lines(x, val_rem))

print('RANDOM EXAMPLES:\n')
print(df.iat[randint(0, 1034), 4])
print('---------')
print(df.iat[randint(0, 1034), 4])

RANDOM EXAMPLES:

type:
permanent
our client, a premier facilities management company, provides tailed integrated fm services and is currently seeking a full-time facilities electrician f a permanent position suppt a client site in cabinteely.
as a leading provider of facilities management services in ireland, europe, and the uk, our client offers a dynamic wk environment where your expertise contributes to the creation of exceptional wkspaces f clients.
wking model: onsite client site in cabinteely
visa sponsship: no
key responsibilities:
provide exemplary customer service, ensuring satisfaction at all times.
adhere to and lead company health and safety policies and guidelines.
comply with and implement existing business operation procedures.
take complete ownership of your w
king environment and systems, ensuring optimal functioning.
ensure service delivery meets
exceeds sla requirements.
manage and ensure timely completion of helpdesk tasks.
maintain high standards of housekeeping t

In [17]:
 # Create the file path for CSV export.
file_path = os.path.join(directory, 'data_jobads_final.csv')
    
# Export the DataFrame to CSV file.
df.to_csv(file_path, index=False)